# Tutorial FFT 3D parallel (MPI)

In this tutorial, we present how to use fluidfft to perform 3D fft in sequential.

Because, we are doing this tutorial in parallel with jupyter and ipyparallel, we first need to create an ipyparallel client and create a direct view as explained [here](http://ipyparallel.readthedocs.io). We previously started an ipcluster with the command `ipcluster start -n 4 --engines=MPIEngineSetLauncher`. This is just a jupyter/ipython thing and it has nothing to do with fluidfft.

In [ ]:
import ipyparallel as ipp
rc = ipp.Client()
dview = rc[:]

Afterwards, we will execute all cells in parallel so we always need to add the magic command `%%px` (see [here](http://ipyparallel.readthedocs.io/en/latest/magics.html))

In [ ]:
%%px
from fluiddyn.util.mpi import rank, nb_proc
print("Hello world! I'm rank {}/{}".format(rank, nb_proc))

In [ ]:
%%px
import numpy as np
from fluidfft.fft3d import methods_mpi
from fluidfft import import_fft_class

In [ ]:
%%px --targets 1
print(methods_mpi)

We import a class and instantiate it:

In [ ]:
%%px
cls = import_fft_class('fft3d.mpi_with_fftw1d')

In [ ]:
%%px
o = cls(4, 8, 12)

Let's have a look at the attribute of this objects.

In [ ]:
%%px --targets 1
print('\n'.join([name for name in dir(o) if not name.startswith('__')]))

Let's run a test and benchmark the fft and ifft functions directly from C++.

In [ ]:
%%px
_ = o.run_tests()

In [ ]:
%%px
results = o.run_benchs()
if rank == 0:
    print('t_fft = {} s; t_ifft = {} s'.format(*results))

Let's understand how the data is stored:

In [ ]:
%%px
print(o.get_dimX_K())

which means that for this class, in Fourier space, the data is transposed...

Now we can get the non dimensional wavenumber in the first and second dimensions:

In [ ]:
%%px
k0, k1, k2 = o.get_k_adim_loc()
print('k0:', k0)
print('k1:', k1)
print('k2:', k2)

In [ ]:
%%px
print(o.get_seq_indices_first_K())

and get the shape of the arrays in real and Fourier space

In [ ]:
%%px
print(o.get_shapeX_seq(), o.get_shapeX_loc())

In [ ]:
%%px
print(o.get_shapeK_seq(), o.get_shapeK_loc())

Now, let's compute fast Fourier transforms. We first initialize arrays:

In [ ]:
%%px
a = np.ones(o.get_shapeX_loc())
a_fft = np.empty(o.get_shapeK_loc(), dtype=np.complex128)

If we do not have the array where to put the result we can do:

In [ ]:
%%px
a_fft = o.fft(a)

If we already have the array where to put the result we can do:

In [ ]:
%%px
o.fft_as_arg(a, a_fft)

And finally for the inverse Fourier transform:

In [ ]:
%%px
a = o.ifft(a_fft)

In [ ]:
%%px
o.ifft_as_arg(a_fft, a)